In [10]:
#This notebook is a tool for splitting and preparing the data from the StrawDI dataset 

#Task 1 : convert the label images to txt files for training with yolov7
#Task2 : extract the bounding boxes from the images and create new ones - for training ripeness and weight models

In [3]:
#Dependencies
import os
import cv2
import numpy as np
import random 

In [15]:
dataset_path = "/Users/larsmoan/Documents/Datasets/StrawDI_Db1/val/label/"
annotations_path = "/Users/larsmoan/Documents/Datasets/StrawDI_Db1/val/annotations/"
#NB! This script will create another copy of the dataset

In [4]:
# Convert Pascal_Voc bb to Yolo
def pascal_voc_to_yolo(x1, y1, x2, y2, image_w, image_h):
    return [((x2 + x1)/(2*image_w)), ((y2 + y1)/(2*image_h)), (x2 - x1)/image_w, (y2 - y1)/image_h]

def yolo_to_pascal_voc(x_center, y_center, w, h,  image_w, image_h):
    w = w * image_w
    h = h * image_h
    x1 = ((2 * x_center * image_w) - w)/2
    y1 = ((2 * y_center * image_h) - h)/2
    x2 = x1 + w
    y2 = y1 + h
    return [x1, y1, x2, y2]

def create_annotations(label_path, label_filename, annotation_path):

    label = cv2.imread(label_path+label_filename)

    #Check if the label is a valid image
    if label is None:
        print("Label is not a valid image")
        return

    #Convert to grayscale
    label_copy = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)


    #Opening txt file for annotations - corresponding name to label
    f = open(annotation_path+label_filename[:-4]+".txt", "w")

    #Getting the maximal value in the grayscale picture - corresponds to number of berries
    max_val = np.amax(label)

    #Doing it for more than one gray value, i.e all the berries
    for i in range(1,max_val+1):
        white_pixels_x = []
        white_pixels_y = []

        """ for row in range(label.shape[0]):
            for col in range(label.shape[1]):
                if label_copy[row][col] == i:
                    white_pixels_y.append(row)
                    white_pixels_x.append(col) """

        #Using numpy to do the search, way to slow with double for loop
        white_pixels_y = np.where(label_copy == i)[0]
        white_pixels_x = np.where(label_copy == i)[1]

       
        
        #True if the lists contains elements
        if white_pixels_x[0] and white_pixels_y[0]:
            x_min = white_pixels_x.min()
            x_max = white_pixels_x.max()
            y_min = white_pixels_y.min()
            y_max = white_pixels_y.max()

            yolo_box = pascal_voc_to_yolo(x_min, y_min, x_max, y_max, label.shape[1], label.shape[0])
        
            #Writing the coordinates to the txt file
            f.write('0' + ' ' + str(yolo_box[0]) + " " + str(yolo_box[1]) + " " + str(yolo_box[2]) + " " + str(yolo_box[3]) + "\n")

In [17]:
#This one creates all the annotations for the dataset - only run it once
for filename in os.listdir(dataset_path):
    create_annotations(dataset_path, filename, annotations_path)

In [13]:
# Task 2 - Create the subpictures that should be used for the second models

img_path = "/Users/larsmoan/Documents/Datasets/StrawDI_Db1/val/img/"
annotations_path = "/Users/larsmoan/Documents/Datasets/StrawDI_Db1/val/annotations/"
sub_images_path = "/Users/larsmoan/Documents/Datasets/StrawDI_Db1/val/sub_images/"
#Getting the boxes from the annotations folder

def create_sub_images(img_path, img_filename, annotations_path, annotation_filename, sub_img_path):
    img = cv2.imread(img_path+img_filename)
    image_w = img.shape[1]
    image_h = img.shape[0]

    #Reading the boxes from the annotations file
    f = open(annotations_path+annotation_filename, "r")
    lines = f.readlines()

    boxes = []
    for line in lines:
        #Convert the boxes to pascal voc
        box = line.split(" ")
        box = box[1:]
        
        #Convert the box to float
        box = [float(i) for i in box]
        boxes.append(box)

    #Creating the sub images
    for i in range(len(boxes)):
        box = boxes[i]
        box = yolo_to_pascal_voc(box[0], box[1], box[2], box[3], image_w, image_h)
        sub_img = img[int(box[1]):int(box[3]), int(box[0]):int(box[2])]

        #Saving the sub images
        cv2.imwrite(sub_img_path+img_filename[:-4]+"_"+str(i)+".png", sub_img)



In [14]:
#This one creates all the sub images for the dataset - only run it once per folder
for img_filename in os.listdir(img_path):
    create_sub_images(img_path, img_filename, annotations_path, img_filename[:-4]+".txt", sub_images_path)